<img src="Dau.png">

#### MASEF, University Paris-Dauphine 2021:   Bryan Delamour 

## Generation of implied volatility surfaces from prices invertion

#### Reference : Deep calibration of rough stochastic volatility models
C. Bayer, B. Stemper

In [1]:
import numpy as np
from cmath import * 
import pickle
import py_vollib
from py_vollib.ref_python.black_scholes.implied_volatility import implied_volatility 

## Parameters

$
\begin{array}{l}
\text { Strikes: }\{0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5\}\\
\text { Maturities: }\{0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0\}
\end{array}
$

In [2]:
nb_samples = 4*10**4
strikes = np.arange(0.5,1.6,0.1)
maturities = np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2])
N_disc = 100 
T_ind = N_disc*maturities
T_ind=T_ind.astype(int)
M = 6*10**4

S0=1

## Loading  Black Scholes prices and rough Bergomi parameters

$
\text { Training set of size } 34.000 \text { and testing set of size } 6.000\\
\text { Rough Bergomi sample: }\left(\xi_{0}, \nu, \rho, H\right) \in \mathcal{U}[0.01,0.16] \times \mathcal{U}[0.5,4.0] \times \mathcal{U}[-0.95,-0.1] \times \mathcal{U}[0.025,0.5]\\
$



In [3]:
# Loading data 

model_parameters_training_File = "model_parameters_training_data.pkl"

with open(model_parameters_training_File, 'rb') as file:  
    model_parameters = pickle.load(file)

XI = model_parameters[:,0]
NU = model_parameters[:,1]
RHO = model_parameters[:,2]
H = model_parameters[:,3]

BS_prices_training_File = "BS_prices_training_data.pkl"

with open(BS_prices_training_File, 'rb') as file:  
    BS_prices = pickle.load(file)

## Implied volatility

$\operatorname{BS}(M, T, \sigma)$ denote the Black-Scholes price of a European Call with moneyness $M$, time to maturity $T$ and assumed constant volatility $\sigma$ of the underlying and let $Q(M, T)$ be the corresponding market price. The BS implied volatility $\sigma_{\mathrm{iv}}(M, T)$ corresponding to $Q(M, T)$ satisfies

$$
Q(M, T)-\mathrm{BS}\left(M, T, \sigma_{\mathrm{iv}}(M, T)\right) \stackrel{!}{=} 0
$$

and the map $(M, T) \mapsto \sigma_{\mathrm{iv}}(M, T)$ is called a volatility surface.

Black-Scholes IVs are inverted from option prices using a publicly available implementation of the implied volatility solver by Jäckel (2015).

In [5]:
def jackel_iv(P,S0,K,T):
    return (implied_volatility(P, S0, K, T, 0, 'c'))
pb = 0 

In [6]:
j_iv = np.zeros(BS_prices.shape)

for i in range(nb_samples):
    if (i%1000 == 0):
        print(i)
    for k in range(len(strikes)):
        for t in range(len(maturities)):
            try :
                j_iv[i,k,t] = jackel_iv( BS_prices[i,k,t], S0, strikes[k], maturities[t])
            except :
                pb +=1
                try:
                    j_iv[i,k,t] = jackel_iv( BS_prices[i,k-1,t], S0, strikes[k], maturities[t])
                    BS_prices[i,k,t] =  BS_prices[i,k-1,t]
                except:
                    j_iv[i,k,t] = jackel_iv( BS_prices[i-1,k,t], S0, strikes[k], maturities[t])
                    BS_prices[i,k,t] =  BS_prices[i-1,k,t]
                    
            

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000


## Data checking and correction

In [8]:
# Data checking and correction 

mean_iv = np.mean(j_iv)
negative = np.sum((j_iv<0)*1)
too_high = np.sum((j_iv>1)*1)
too_small = np.sum((j_iv<10**-6))

print("negative", negative)
print("too high", too_high)
print("too small", too_small)


ax_1 = np.where(j_iv>1)[0]
ax_2 = np.where(j_iv>1)[1]
ax_3 = np.where(j_iv>1)[2]
n = len(np.where(j_iv>1)[0])



negative 0
too high 158
too small 1505


In [9]:
## too high correction with mean over the respective (k,t)

ax_1 = np.where(j_iv>1)[0]
ax_2 = np.where(j_iv>1)[1]
ax_3 = np.where(j_iv>1)[2]
n = len(np.where(j_iv>1)[0])

for i in range(n):
    j_iv[ax_1[i],ax_2[i],ax_3[i]] = np.mean(j_iv[:,ax_2[i],ax_3[i]])
    
ax_1 = np.where(j_iv<10**-6)[0]
ax_2 = np.where(j_iv<10**-6)[1]
ax_3 = np.where(j_iv<10**-6)[2]
n = len(np.where(j_iv<10**-6)[0])

# same with too small values
for i in range(n):
    j_iv[ax_1[i],ax_2[i],ax_3[i]] = np.mean(j_iv[:,ax_2[i],ax_3[i]])

In [10]:
## Double check

too_high = np.sum((j_iv>1)*1)
too_small = np.sum((j_iv<10**-6))

print(too_high)
print(too_small)

0
0


## Saving implied volatility surfaces

In [11]:
## Saving implied volatility surfaces

iv_training_File = "iv_training_data.pkl"  

with open(iv_training_File, 'wb') as file:
    pickle.dump(j_iv, file)